In [4]:
import causaldata
import numpy as np
print(dir(causaldata))
df = causaldata.black_politicians.load_pandas().data
print(causaldata.black_politicians.NOTE)
print(causaldata.black_politicians.DESCRLONG)
print(causaldata.black_politicians.SOURCE)

['Mroz', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'abortion', 'adult_services', 'auto', 'avocado', 'black_politicians', 'castle', 'close_elections_lmb', 'cps_mixtape', 'credit_cards', 'gapminder', 'google_stock', 'gov_transfers', 'gov_transfers_density', 'greek_data', 'mortgages', 'nhefs', 'nhefs_codebook', 'nhefs_complete', 'nsw_mixtape', 'organ_donations', 'restaurant_inspections', 'ri', 'scorecard', 'snow', 'social_insure', 'texas', 'thornton_hiv', 'titanic', 'training_bias_reduction', 'training_example', 'yule']
::
    Number of observations - 5593
    Number of variables - 14
    Variables name definitions::
            leg_black - Legislator receiving email is Black
            treat_out - Email is from out-of-district
            responded - Legislator responded to email
            totalpop - District population
            medianhhincom - District median household income
            black_m

In [5]:
outcome = 'responded'
treatment = 'treat_out'
rest = df.drop([outcome, treatment, 'leg_black'], axis=1).columns

In [6]:
#import wooldridge
#df = wooldridge.data('jtrain3')
#df['avg'] = 0.5 * (df.re74+df.re75)
#df = df.dropna()
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest]
print(df.shape)
df.head()

(5593, 14)


,leg_black,treat_out,responded,totalpop,medianhhincom,black_medianhh,white_medianhh,blackpercent,statessquireindex,nonblacknonwhite,urbanpercent,leg_senator,leg_democrat,south
0,0,0,0,1.5873,5.0625,2.6814,2.6586,0.007119,0.227,0,0.695601,0,0,0
1,0,0,1,1.6218,4.9713,2.7126,2.6619,0.005796,0.227,0,0.618073,0,0,0
2,0,0,1,1.6710,6.9646,2.3087,2.9973,0.012029,0.227,0,0.824331,0,0,0
3,0,0,1,1.6122,4.1811,2.4668,2.4887,0.004280,0.227,1,0.000000,0,0,0
4,0,1,1,1.5622,3.1152,2.1490,2.0597,0.008258,0.227,1,0.000000,0,1,0


In [7]:
# Simple Comparision of Means
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:              responded   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     437.4
Date:                Sun, 04 Dec 2022   Prob (F-statistic):           1.44e-93
Time:                        20:44:37   Log-Likelihood:                -3781.3
No. Observations:                5593   AIC:                             7567.
Df Residuals:                    5591   BIC:                             7580.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.2661      0.013    -20.913      0.0

In [8]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:              responded   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     45.56
Date:                Sun, 04 Dec 2022   Prob (F-statistic):          4.41e-104
Time:                        20:44:40   Log-Likelihood:                -3730.6
No. Observations:                5593   AIC:                             7487.
Df Residuals:                    5580   BIC:                             7573.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.2673      0.013    -21.163      0.0

In [9]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=rest)
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

TypeError: The covariates x_cols must be of str or list type (or None). Index(['totalpop', 'medianhhincom', 'black_medianhh', 'white_medianhh',
       'blackpercent', 'statessquireindex', 'nonblacknonwhite', 'urbanpercent',
       'leg_senator', 'leg_democrat', 'south'],
      dtype='object') of type <class 'pandas.core.indexes.base.Index'> was passed.

In [10]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,-0.268,0.013,-21.141,0.0,-0.293,-0.243
